In [1]:
import folium
import warnings
import pandas as pd
import matplotlib.pyplot as graph
from itertools import groupby
from folium.plugins import MarkerCluster
from Helpers.phenotypes import load_phenotypes
warnings.filterwarnings('ignore')

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [2]:
def dd_to_dms(dd):
    is_positive = dd >= 0
    dd = abs(dd)
    minutes, seconds = divmod(dd * 3600, 60)
    degrees, minutes = divmod(minutes, 60)
    degrees = degrees if is_positive else -degrees
    return (int(degrees), int(minutes), round(seconds, 2))


def dms_to_dd(dms):
    d, m, s = dms.replace('(', '').replace(')', '').split(', ')
    dd = int(d) + float(m)/60 + float(s)/3600
    return dd

In [3]:
phen, _ = load_phenotypes('Data/beeomics_phenotypes_2018-05-16.csv')

with open('merged-925-R2-filtered-0.2af-0.0missing-1step.pvcf') as pvcf:
    genotyped_colonies = next(pvcf).strip().split('\t')[4:]
    
phen = phen[phen['colony_id'].isin(genotyped_colonies)]

df = pd.read_excel('yard-gps-data-merged.xlsx')
df.columns = [col.lower() for col in df.columns]
df.head()

Processing Data/beeomics_phenotypes_2018-05-16.csv
Hygiene Normed (N:844) -> [0.1899400260866712, 1.5707963267948966]
Honey Production Normed (N:339) -> [-2.63, 97.38]
Varroa Growth Rate (N:784) -> [0.17722954771019425, 17.175]
Weight Gain (N:712) -> [-0.26074498567335247, 0.5101969872537658]
Instant Weight Gain (N:712) -> [-13.4, 48.45]


,yard,coordinates
0,Peter Awram,"49.021085, -122.393534"
1,Steve Blenkhorn,"49.189212, -122.253766"
2,Judy & Mike Campbell,"49.049181, -122.4535087"
3,Heather Higo,"49.0205654, -122.5693326"
4,Deb Tin Tun,"49.147440, -122.696666"


# Format Lat Longs

In [4]:
dms = df[df['coordinates'].str.contains('°')]
dd = df[~(df['coordinates'].str.contains('°'))]

# Format DD
dd['coordinates'] = dd['coordinates'].str.replace('N 46,', 'N 46.').str.replace('W -71,', 'W -71.')
dd['coordinates'] = dd['coordinates'].str.replace('N ', '').str.replace('W ', ', ')

dd[['lat_dd', 'long_dd']] = dd['coordinates'].str.split(',', expand=True)
dd['lat_dd'] = pd.to_numeric(dd['lat_dd'].str.strip())
dd['long_dd'] = pd.to_numeric(dd['long_dd'].str.strip())

dd['lat'] = dd['lat_dd'].apply(lambda lat_i: dd_to_dms(lat_i))
dd['long'] = dd['long_dd'].apply(lambda long_i: dd_to_dms(long_i))

# Format DMS
dms['coordinates'] = dms['coordinates'].str.replace('°', ', ').str.replace("'", ', ').str.replace('"', ')')
dms['coordinates'] = dms['coordinates'].str.replace('W', '').str.replace('N,', 'N')
dms[['lat', 'long']] = dms['coordinates'].str.split('N ', expand=True)
dms['lat'] = '(' + dms['lat']
dms['long'] = '(' + dms['long']

dms['lat_dd'] = dms['lat'].apply(lambda x: dms_to_dd(x))
dms['long_dd'] = dms['long'].apply(lambda x: dms_to_dd(x)) * -1

In [5]:
df = pd.concat([dd, dms], ignore_index=True)
df.to_csv('yard-lat-longs+weather-station-id.csv', index=False)

In [6]:
df.head()

,coordinates,lat,lat_dd,long,long_dd,yard
0,"49.021085, -122.393534","(49, 1, 15.91)",49.021085,"(-122, 23, 36.72)",-122.393534,Peter Awram
1,"49.189212, -122.253766","(49, 11, 21.16)",49.189212,"(-122, 15, 13.56)",-122.253766,Steve Blenkhorn
2,"49.049181, -122.4535087","(49, 2, 57.05)",49.049181,"(-122, 27, 12.63)",-122.453509,Judy & Mike Campbell
3,"49.0205654, -122.5693326","(49, 1, 14.04)",49.020565,"(-122, 34, 9.6)",-122.569333,Heather Higo
4,"49.147440, -122.696666","(49, 8, 50.78)",49.147440,"(-122, 41, 48.0)",-122.696666,Deb Tin Tun


In [7]:
df.to_csv('beeomics-yard-lat-longs.csv', index=False)

In [8]:
yards_to_yards = {
    'Peter Awram': 'Awram',
    'Steve Blenkhorn': 'Steve',
    'Judy & Mike Campbell': 'Campbell',
    'Heather Higo': 'Higo',
    'Deb Tin Tun': 'Deb',
    'Scott Gordon': 'Scott',
    'Nancy Leonard': 'Nancy',
    'Brenda Jager': 'Brenda',
    'Stan Reist': 'Stan',
    'Jamie McDonald': 'Jamie McD',
    'Liz Huxter (Home yard)': 'Liz Huxter',
    'Liz Huxter (Winter yard)': 'Liz Huxter',
    'Leroy Harder': 'Leroy',
    'Overwinter location for yards 2, 3, 4': '2',
    '2': '2',
    '3': '3',
    '4': '4',
    'Ohler ': 'Ohler',
    'Heidmiller': 'Heidmiller',
    'Perlich ': 'Perlich',
    'Main  ': 'Main',
    'Overwinter location for Ohler and Heidmiller': 'Ohler',
    'Overwinter location for Perlich and Main': 'Perlich',
    'OBS': 'OBS',
    'HOG': 'HOG', 
    'CW': 'CW',
    'ARB': 'ARB', 
    'Bee': 'Bee', 
    'Thurston': 'Thurston', 
    'Horton': 'Horton',
    'Breeder': 'Breeder', 
    'Home-CA': 'Home -CA', 
    'Home-DA': 'Home -DA',
    'Rowanwood': 'Rowanwood', 
    'Albion': 'Albion',
    'Italian Guy': 'Italian Guy', 
    'Home-FB': 'Home -FB', 
    'Queensville': 'Queensville',
    'Neighsmith': 'Neighsmith', 
    'Hamilton': 'Hamilton', 
    'Home-FC': 'Home - FC', 
    'Hillside': 'Hillside', 
    'Home-HB': 'Home -HB',
    'Willowbee': 'Willowbee', 
    'Home-JA': 'Home -JA', 
    'Home-KA': 'Home -KA', 
    'McClouds': "McCloud's", 
    'Goyer': 'Goyer', 
    'Arbor': 'Arbor',
    'HQ': 'HQ', 
    'Côté': 'Côté', 
    'Langlois': 'Langlois', 
    'Martel': 'Martel', 
    'Martel (new)': 'Martel', 
    'Bérube': 'Bérubé',
    'Anicet': 'Anicet',
    'Regalbuto': 'Regalbuto',
}

In [9]:
def ranges(i):
    for a, b in groupby(enumerate(i), lambda x: x[0] - int(x[1])):
        b = list(b)
        yield b[0][1], b[-1][1]
        
def plot_map(df, cluster=True):
    m = folium.Map(location=[50.372, -100.6972], zoom_start=4, tiles='Stamen Terrain')
    
    marker_cluster = MarkerCluster().add_to(m)
        
    for yard, lat, long in zip(df['yard'], df['lat_dd'], df['long_dd']):
        yard = str(yard).replace("'", '')
        colonies = phen[phen['yard'] == yards_to_yards[yard]].index.tolist()
        popup = f'<h4>{yard}: {len(colonies)} Colonies</h4>'
        popup += '<ul>'
        
        for start, stop in ranges(colonies):
            if start == stop:
                popup += f'<li>{start}'
            else:
                popup += f'<li>{start}-{stop}</li>'
        popup += '</ul>'
        folium.Marker([lat, long], popup=popup).add_to(marker_cluster)
    display(m)

plot_map(pd.concat([dd, dms]))

In [10]:
def plot_map(df, cluster=True):
    m = folium.Map(location=[50.372, -100.6972], zoom_start=4, tiles='Stamen Terrain')
    
    marker_cluster = MarkerCluster().add_to(m)
        
    for yard, lat, long in zip(df['yard'], df['lat_dd'], df['long_dd']):
        yard = str(yard).replace("'", '')
        
        # Don't include where colonies overwinter
        if 'winter' in yard.lower() or 'new' in yard.lower():
            continue
        
        colonies = phen[phen['yard'] == yards_to_yards[yard]].index.tolist()
        
        for _ in colonies:
            folium.Marker([lat, long]).add_to(marker_cluster)
    display(m)

plot_map(pd.concat([dd, dms]))